# Testing of the model

In [10]:
import pandas as pd

In [11]:
# Loading saved model
from xgboost.sklearn import XGBRegressor
regressor = XGBRegressor(n_jobs=-1)
regressor.load_model('./ModelFiles/RealEstate_PARIS_FR_2022.xgbmodel')

In [12]:
# Loading saved preprocessor
from pickle import load
preprocessor = load(open('./ModelFiles/preprocessor.dmp', 'rb'))

In [13]:
# Building tests DF
df_real_test = pd.DataFrame({'Nature mutation':['Vente', 'Vente', 'Vente',  "Vente",  "Vente"]
                             , 'Type local':['Appartement', 'Maison', 'Appartement', 'Appartement', 'Appartement']
                             , 'Nombre pieces principales': [5, 3, 6, 3, 4]
                             , 'Surface': [107, 60, 179, 83, 110]
                             , 'Adresse':['RUE MONGE 75005 PARIS FRANCE', "Rue de l'Estrapade 75005 PARIS FRANCE", 'Madeleine 75008 PARIS FRANCE', '75017 PARIS FRANCE', 'Place Vendome 75001 PARIS FRANCE']})

In [14]:
df_real_test

,Nature mutation,Type local,Nombre pieces principales,Surface,Adresse
0,Vente,Appartement,5,107,RUE MONGE 75005 PARIS FRANCE
1,Vente,Maison,3,60,Rue de l'Estrapade 75005 PARIS FRANCE
2,Vente,Appartement,6,179,Madeleine 75008 PARIS FRANCE
3,Vente,Appartement,3,83,75017 PARIS FRANCE
4,Vente,Appartement,4,110,Place Vendome 75001 PARIS FRANCE


In [15]:
# Computing GPS coordonates for testing
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="MyAppRE75")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df_real_test['location'] = df_real_test['Adresse'].apply(geocode)
df_real_test['Latitude'] = df_real_test['location'].apply(lambda loc: loc.point[0] if loc else None)
df_real_test['Longitude'] = df_real_test['location'].apply(lambda loc: loc.point[1] if loc else None)
df_real_test.drop(columns=['Adresse', 'location'], axis=1, inplace=True)
df_real_test

,Nature mutation,Type local,Nombre pieces principales,Surface,Latitude,Longitude
0,Vente,Appartement,5,107,48.847810,2.350713
1,Vente,Maison,3,60,48.844558,2.347557
2,Vente,Appartement,6,179,48.871027,2.319279
3,Vente,Appartement,3,83,48.885676,2.308889
4,Vente,Appartement,4,110,48.867463,2.329428


In [16]:
# Transforming tests
X_real_test = preprocessor.transform(df_real_test)
# Predicting target value
Y_real_test = regressor.predict(X_real_test)

# Expected values : 
# 1 300 000 : https://www.logic-immo.com/detail-vente-7be64076-668e-fc49-6abb-18b74ab0a503.htm?mea=extend-boost-3
# 1 200 000 : https://www.logic-immo.com/detail-vente-84f94262-d46f-f5e7-10e4-babc1d4476dc.htm?mea=extend-boost-3
# 2 850 000 : https://www.logic-immo.com/detail-vente-fe46a133-b06a-422f-8fb3-b05e0b0f816d.htm?mea=extend-boost-3
#   920 000 : https://www.logic-immo.com/detail-vente-3c97ccf4-c50a-d3c8-5a7b-d5592979ff62.htm?mea=extend-boost-3
# 1 825 000 : https://www.logic-immo.com/detail-vente-431b3570-2302-80f9-3bf6-480615bf146e.htm?mea=extend-boost-3

In [17]:
for i in Y_real_test:
    print('{:,} €'.format(round(i)).replace(',', ' '))

1 433 078 €
1 183 345 €
2 202 865 €
789 806 €
1 492 349 €
